In [1]:
# import libraries
import io
import os
import sys
import uuid

import google.generativeai as genai
import gradio as gr

from PIL import Image
from dotenv import load_dotenv

sys.path.append(os.path.abspath(r"./utilities"))
import MovieScriptGenerator
import MoviePosterGenerator

c:\Users\hknal\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# temporary directory to save images that the user uploads
SAVE_DIR = "temp_images"
os.makedirs(SAVE_DIR, exist_ok=True)

In [3]:
def generate(image):
    """
    Processes an input image to generate a movie script and a resized movie poster.

    Args:
        image (PIL.Image.Image): The input image.

    Returns:
        tuple: (str, PIL.Image.Image)
            - The generated movie script as a string.
            - The resized movie poster as a PIL.Image.Image object.
    """
    file_path = os.path.join(SAVE_DIR, f"{uuid.uuid4().hex}.jpg")
    image.save(file_path)
    movieScriptGenerator = MovieScriptGenerator.MovieScriptGenerator()
    script = movieScriptGenerator.generate_script(file_path)
    moviePosterGenerator = MoviePosterGenerator.MoviePosterGenerator()
    poster = moviePosterGenerator.generate_poster(script)
    new_size = (300, 300)
    resized_poster = poster.resize(new_size, Image.LANCZOS)
    return script, resized_poster

In [4]:
# Create a Gradio interface for generating a movie script and poster from an image

app = gr.Interface(
    fn=generate,
    inputs=gr.Image(type="pil"), 
    outputs=[gr.Textbox(label="Movie information"), 
             gr.Image(type="pil", label="Movie Poster")],  
    title="Movie Script Generator and Poster Generator",
    description="Paste or upload an image. The system will generate movie information and create a poster.")
# Launch the app
app.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://f7262ec1c09fd7b716.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [5]:
# Version 2: Gradio UI with Blocks for better layout

with gr.Blocks() as demo:
    # 🎬 Title
    gr.Markdown("## 🎥 Movie Script & Poster Generator 🎭")
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("### Upload an Image 🎨")
            image_input = gr.Image(type="pil", label="Paste or Upload Image")  
            generate_button = gr.Button("🎬 Generate Movie")  # Added button for better UX
                
    with gr.Row():
        with gr.Column():
            gr.Markdown("### 📜 Generated Movie Script")
            text_output = gr.Textbox(label="Movie Information", interactive=False)
        
        with gr.Column():
            gr.Markdown("### 🎨 Generated Movie Poster")
            image_output = gr.Image(type="pil", label="Movie Poster")

    generate_button.click(fn=generate, inputs=image_input, outputs=[text_output, image_output])

# Launch the UI
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://87ceb9e04bf096062f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
